In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 transformers[sentencepiece]==4.31.0 evaluate bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
    HfArgumentParser,
    LlamaTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [3]:
if torch.cuda.is_available():
    device = torch.device(0)
    device_map = {"": 0}
else:
    device = torch.device('cpu')
    device_map = "cpu"

print(device)

cuda:0


In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "chenqile09/chinese-alpaca-2-LoRA-13B-couplet-100k"

# The instruction dataset to use
dataset_name = "chenqile09/llama2-chinese-couplet-100k"

In [5]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [6]:
# Load dataset
dataset = load_dataset(dataset_name, split="validation")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [7]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

torch.float16


In [8]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Load tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


## Set up eval configs

In [10]:
from tqdm import tqdm
import evaluate
import numpy as np

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [11]:
# prompt generation and helper function to generate output
COUPLET_SYSTEM_PROMPT = """你是一个写对联的助手，你会根据给出的上联来写下联。上联和下联的字数要一致，并且平仄要对应。"""

TEMPLATE = (
    "[INST] <<SYS>> {system_prompt} <</SYS>> {instruction} [/INST]"
)

def generate_prompt(instructions, system_prompt=COUPLET_SYSTEM_PROMPT):
    return [TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt}) for instruction in instructions]

def generate_output_from_llm(model, tokenizer, generation_config, instructions, system_prompt=COUPLET_SYSTEM_PROMPT):

    input_texts = generate_prompt(instructions, system_prompt)
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True)
    generation_outputs = model.generate(
        input_ids = inputs["input_ids"].to(device),
        attention_mask = inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config
    )

    outputs = [tokenizer.decode(s, skip_special_tokens=True) for s in generation_outputs]
    responses = [output.split("[/INST]")[-1].strip() for output in outputs]
    return responses

In [12]:
# Set up generation config
generation_config = GenerationConfig(
    temperature=0.2,
    top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.1,
    max_new_tokens=400
)

## Run text generation on a few inputs

In [13]:
# some example inputs
# https://github.com/minaxixi/ai-couplet
instructions = [
    "欲把心事付瑶琴",
    "半衾幽梦香初散",
    "一曲清音韵未央",
    "应是飞鸿踏泥雪",
    "云破月来花弄影",
    "别后相思空一水",
    "书中自有黄金屋",
    "花谢花飞花满天",
    "杨柳岸晓风残月",
    "上海自来水来自海上",
]

In [14]:
# Generate responses in batch, and then print out

responses = generate_output_from_llm(
    model,
    tokenizer,
    generation_config,
    instructions,
    COUPLET_SYSTEM_PROMPT
)

for instruction, response in zip(instructions, responses):
    print(instruction+';'+response)

欲把心事付瑶琴;不教秋水洗尘埃
半衾幽梦香初散;一枕相思泪已干
一曲清音韵未央;千丝细雨润无声
应是飞鸿踏泥雪;岂能白马过河沙
云破月来花弄影;风清日丽柳摇金
别后相思空一水;归来寂寞满三杯
书中自有黄金屋;梦里常闻玉笛声
花谢花飞花满天;月圆月缺月盈池
杨柳岸晓风残月;荷花池晚雨清波
上海自来水来自海上;天安门广场地安天安


## Run eval on the validation dataset

In [15]:
# batch the dataset into the dataloader
ds_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64, # tunable based on the GPU VRAM. 64 is good for 16GB VRAM (V100)
    num_workers=4,
    pin_memory=True,
)

In [16]:
# eval mode
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(55296, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [17]:
# run eval on bert-score
bertscore = evaluate.load("bertscore")

for i, batch in tqdm(list(enumerate(ds_loader))):
    with torch.no_grad():
        predictions = generate_output_from_llm(
            model,
            tokenizer,
            generation_config,
            batch['input'],
            COUPLET_SYSTEM_PROMPT
        )

    bertscore.add_batch(references=batch['output'], predictions=predictions)

result = bertscore.compute(lang='zh')

100%|██████████| 250/250 [08:41<00:00,  2.08s/it]


In [19]:
# print the result
results = dict()

for key, val in result.items():
    if key in {'precision', 'recall', 'f1'}:
        results[key] = np.mean(val)

print(results)

{'precision': 0.662040510199964, 'recall': 0.6615949584394694, 'f1': 0.6617958773002028}
